In [1]:
#Predict reactivity of a protein-ligand complex
#Before running, download results directory into pred/script/pafnucy/
    #https://gitlab.com/cheminfIBB/pafnucy/-/tree/master/results 
import os
import sys
import pandas as pd
import numpy as np
import pickle
from openbabel import pybel

os.chdir('/home/azamh/demo/seq_struct_func/pred')

In [2]:
#Arguments
protein = 'tropb'
ligand = '2'
pdb_with_fad_path = f'../cofactor/pdb_with_fad/{protein}_fad.pdb'
dockdir = f'../dock/poses/{protein}_{ligand}_prot'
stereo_df = pd.read_excel(f'stereo/{protein}_{ligand}_stereo.xlsx', header = 0, index_col = 0)

In [3]:
#Read trained logistic regressor model for consensus reactivity
log_reg = pickle.load(open(f'../si_data/log_reg_models/prot_0.75_fftdock_3_logreg.pkl', 'rb'))
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Conversion   No. Observations:                   67
Model:                          Logit   Df Residuals:                       63
Method:                           MLE   Df Model:                            3
Date:                Sun, 11 Dec 2022   Pseudo R-squ.:                  0.2368
Time:                        22:19:50   Log-Likelihood:                -34.980
converged:                       True   LL-Null:                       -45.835
Covariance Type:            nonrobust   LLR p-value:                 7.501e-05
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Avg Fad Angle                     0.1035      0.043      2.393      0.017       0.019       0.188
Avg Anion Dist                   -0.4565      0.199     -2.293      0.022      -0.847      -0.066
Avg Docking Energy Efficiency    -1.3699      0.389     -3.519      0.000      -2.133      -0.607
Avg Pafnucy pKa Efficiency      -16.6168      5.488     -3.028      0.002     -27.373      -5.860
=================================================================================================
"""

In [4]:
#Angle, anion dist and docking energy from stereochemistry analysis of cluster df
stereo_df

,cluster,size,min_ener,min_index,average energy,std energy,stereo,angle,dist_to_fad,anion_dist
0,"['105', '135', '147', '162', '179', '199', '22...",8,-50.03,224,-28.72,15.96,R,131.45,4.412384,3.315454
1,"['108', '115', '185', '213', '233', '242', '25...",17,-46.84,185,-20.94,17.58,R,113.79,5.454536,3.451893
2,['145'],1,-42.09,145,-42.09,0.00,R,115.82,5.454525,4.728620
3,"['122', '128', '136', '204', '211', '261', '28...",11,-40.18,211,15.15,30.23,R,110.16,5.373566,4.580234
4,"['194', '241', '438', '458', '471', '63']",6,-38.91,438,-33.32,3.23,R,118.84,5.102235,4.227627
...,...,...,...,...,...,...,...,...,...,...
103,['245'],1,1866.75,245,1866.75,0.00,R,116.61,7.197409,5.214141
104,['287'],1,3351.10,287,3351.10,0.00,S,61.76,2.704134,7.536139
105,"['156', '215']",2,3791.10,156,3799.87,8.77,S,30.87,2.076857,8.937882
106,['146'],1,3817.38,146,3817.38,0.00,R,157.18,1.887237,8.958015


In [5]:
#Get predicted pKd from Pafnucy
#Use top docked pose for protein-ligand complex
top_index = stereo_df['min_index'].iloc[0]
top_pose_path = os.path.join(dockdir, f'{protein}_{ligand}_{top_index}.pdb')

In [6]:
#Prepare protein ligand complex
complex_file = f'pafnucy/{protein}_{ligand}.hdf'
prepare_cmd = f'{sys.executable} script/pafnucy/prepare.py -l {top_pose_path} --ligand_format pdb ' + \
f'-p {pdb_with_fad_path} --pocket_format pdb -o {complex_file}'
os.system(prepare_cmd)

1 ligands and 1 pockets to prepare:
 pocket: /home/azamh/demo/seq_struct_func/cofactor/pdb_with_fad/tropb_fad.pdb
 ligand: /home/azamh/demo/seq_struct_func/dock/poses/tropb_2_prot/tropb_2_224.pdb



reading /home/azamh/demo/seq_struct_func/dock/poses/tropb_2_prot/tropb_2_224.pdb


created /home/azamh/demo/seq_struct_func/pred/pafnucy/tropb_2.hdf with 1 structures


0

In [7]:
#Predict with trained CNN
prediction_file = f'pafnucy/{protein}_{ligand}.csv'
network = 'script/pafnucy/results/batch5-2017-06-05T07:58:47-best'
predict_cmd = f'{sys.executable} script/pafnucy/predict.py -i {complex_file} -n {network} -o {prediction_file}'
os.system(predict_cmd)

2022-12-11 22:19:53.321105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 22:19:53.442820: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Instructions for updating:
non-resource variables are not supported in the long term
2022-12-11 22:19:58.623212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  

loaded 1 complexes

restored network from /home/azamh/demo/seq_struct_func/pred/script/pafnucy/results/batch5-2017-06-05T07:58:47-best

20 samples per batch

results saved to /home/azamh/demo/seq_struct_func/pred/pafnucy/tropb_2.csv


0

In [8]:
pred_pkd = pd.read_csv(prediction_file, header = 0, index_col = 0)['prediction'].iloc[0]
pred_pkd

5.870636

In [9]:
#Process values for logistic regression model

#Dictionary for ligand heavy atom counts:
ha_dict = {'2':17, '3':18, '4':17, '5':14}

#Read in pose geometric information from stereo_df
top_cluster = stereo_df.iloc[0]
dock_energy = top_cluster['min_ener']
angle = top_cluster['angle']
if angle > 90:
    angle = 180 - angle
anion_dist = top_cluster['anion_dist']

#Rescale by heavy atom counts for pkd and dock energy
dock_energy_efficiency = dock_energy / ha_dict[ligand]
pred_pkd_efficiency = pred_pkd / ha_dict[ligand]

print('dock_energy_efficiency', dock_energy_efficiency, 'kcal/mol/HA')
print('angle', angle, 'degrees')
print('anion_dist', anion_dist, 'angstrom')
print('pred_pkd_efficiency', pred_pkd_efficiency, '1/HA')

dock_energy_efficiency -2.942941176470588 kcal/mol/HA
angle 48.55000000000001 degrees
anion_dist 3.315454418326393 angstrom
pred_pkd_efficiency 0.3453315294117647 1/HA


In [10]:
#Predict with logistic regression model: 0 -> unreactive, 1 -> reactive
log_reg.predict([angle, anion_dist, dock_energy_efficiency, pred_pkd_efficiency])

array([0.85891097])